Объявление нужных нам модулей и классов.

In [1]:
from random import randint

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

Создание DataFrame с именованием колонок.

In [5]:
df_start = pd.read_csv("task_1.csv", sep = ";", encoding="windows-1251", header=None)
df_start

IndexError: single positional indexer is out-of-bounds

Добавление колонки label, хранящая метку класса(0 для котов, 1 для собак).

In [ ]:
my_series["label"] = 0

for i, class_ in enumerate(my_series["class"]):
    if class_ == "dog":
        my_series.iloc[[i], [2]] = 1

my_series

Добавление колонок с шириной, высотой и глубиной каждого изображения.

In [ ]:
my_series["height"] = 0
my_series["width"] = 0
my_series["depth"] = 0

for i,path in enumerate(my_series["abspath"]):
    data = cv2.imread(path[-20:]).shape
    my_series.iloc[[i], [3]] = data[0]
    my_series.iloc[[i], [4]] = data[1]
    my_series.iloc[[i], [5]] = data[2]

my_series

Статистические данные о DataFrame(данные сбалансированы).

In [ ]:
my_series.describe()

Функция, возвращающая DataFrame, отфильтрованная по метке класса.

In [54]:
def part_5(my_series: pd.core.series.Series,
           label: int) -> pd.core.series.Series:
    return my_series[my_series["label"] == label]

label_my_series = part_5(my_series, 1)
label_my_series.to_csv("label_my_series.csv", sep=" ")

Функция, возвращающая отфильтрованный DataFrame по метке, высоте и ширине.

In [55]:
def part_6(my_series: pd.core.series.Series, label: int,
           h: int, w: int) -> pd.core.series.Series:
    tmp_1 = my_series[my_series["label"] == label]
    tmp_2 = tmp_1[tmp_1["height"] <= h]
    return tmp_2[tmp_2["width"] <= w]

image_my_series = part_6(my_series, 0, 319, 500)
image_my_series.to_csv("image_my_series.csv", sep=" ")

Функция, вычисляющая кол-во пикселей для исходного DataFrame и макс., мин. и ср. значения пикселей для DataFrame, отфильтрованного по метке.

In [ ]:
def part_7(label: int) -> None:
    my_series["pixels"] = (my_series["height"] *
                           my_series["width"]  *
                           my_series["depth"])

    pix_my_series = part_5(my_series, label)
    
    print("The maximum number of pixels is ", pix_my_series["pixels"].max())
    print("The minimum number of pixels is", pix_my_series["pixels"].min())
    print("The average number of pixels is", pix_my_series["pixels"].mean())

part_7(0)

Функция, строящая гистограмму.

In [57]:
def part_8(my_series: pd.core.series.Series, label: int) -> np.ndarray:
    label_series = my_series[my_series["label"] == label]
    rand_num = randint(0, label_series["abspath"].count())
    abspath = my_series.iloc[rand_num, 1]

    image = cv2.imread(abspath[-20:])

    histr_1 = cv2.calcHist([image], [0], None, [256], [0,256])
    histr_2 = cv2.calcHist([image], [1], None, [256], [0,256])
    histr_3 = cv2.calcHist([image], [2], None, [256], [0,256])
    
    return histr_1, histr_2, histr_3

Отрисовка гистограмм.

In [ ]:
histr_1, histr_2, histr_3 = part_8(my_series, 0)
col = ('b','g','r')
plt.plot(histr_1, color=col[0])
plt.plot(histr_2, color=col[1])
plt.plot(histr_3, color=col[2])
plt.xlabel("value")
plt.ylabel("count")
plt.title("histogram")
plt.xlim([0,256])
plt.show()

Создание электронной таблицы, куда загружаем весь наш исходный DataFrame.

In [59]:
my_series.to_csv("my_series.csv", sep=" ")